In [ ]:
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, Normalizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import roc_auc_score
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import mutual_info_regression
from datetime import datetime

%matplotlib inline
sns.set()

In [ ]:
conn = sqlite3.connect('../Data/db/fraude_detection_warehouse_.db')

alert = pd.read_sql_query("SELECT * FROM alerts", conn)
customers = pd.read_sql_query("SELECT * FROM customers", conn)
devices = pd.read_sql_query("SELECT * FROM devices", conn)
transaction_history = pd.read_sql_query("SELECT * FROM transaction_history", conn)
transaction_patterns= pd.read_sql_query("SELECT * FROM transaction_patterns", conn)
transactions = pd.read_sql_query("SELECT * FROM transactions", conn)
transactions.head()

In [ ]:
# Fusion des tables
df1 = transactions.merge(customers, on='customer_id', how='left').drop(columns=['first_name', 'last_name', 'transaction_date'])
df2 = df1.merge(devices, on='device_id', how='left')
df3 = df2.merge(transaction_history, on=['customer_id', 'transaction_id'], how='left')

df3.head()

In [ ]:
df3.info()

In [ ]:
date_columns = ["registration_date", "date_of_birth", "transaction_date", "last_used"]

for column in date_columns:
    df3[column] = pd.to_datetime(df3[column])

df3.info()  

In [ ]:
data = df3.copy()
data.drop(columns=["transaction_id", "customer_id", "device_id", "email", "phone_number", "address", "history_id"], inplace=True)
data.head()

## Feature engineering

In [ ]:
scaler = StandardScaler()
num_columns = list(data.select_dtypes(["float64"]).columns)
for col in num_columns:
    data[col] = scaler.fit_transform(data[[col]])

In [ ]:
# Encodage des variables categorielles
le = LabelEncoder()
categorical_columns = data.select_dtypes(include=['object']).columns
for col in categorical_columns:
    data[col] = le.fit_transform(data[col].astype(str))

In [ ]:
data.info()

In [ ]:
# Fonction pour extraire les composantes de date
def extract_date_features(data, column):
    data[f'{column}_day'] = data[column].dt.day
    data[f'{column}_month'] = data[column].dt.month
    data[f'{column}_year'] = data[column].dt.year
    data[f'{column}_weekday'] = data[column].dt.weekday
    data[f'{column}_quarter'] = data[column].dt.quarter
    return data

# Conversion des dates et extraction des caractéristiques
date_columns = ['transaction_date','last_used', 'registration_date'] # 'date_of_birth'
for col in date_columns:
    data = extract_date_features(data, col)
    
    # Suppression de la colonne de date originale
    data = data.drop(columns=[col])

In [ ]:
print(data.columns)

In [ ]:
# Sélectionner uniquement les colonnes de date nouvellement créées
date_columns = [col for col in data.columns if any(x in col for x in ['_day', '_month', '_year', '_weekday', '_quarter'])]

# Afficher les premières lignes de ces colonnes
print(data[date_columns].head(10))

In [ ]:
date_of_today = datetime.now()

def calculate_and_assign(row):
  years_since_birth = int((date_of_today - row['date_of_birth']).days / 360)
  return pd.Series({'year_since_birth': years_since_birth})

data["year_since_birth"] = data.apply(calculate_and_assign, axis=1)
data.drop(columns=["date_of_birth"], inplace=True)
data.head()

In [ ]:
lignes_avec_manquantes = df3[df3.isnull().any(axis=1)]

nombre_lignes_manquantes = lignes_avec_manquantes.shape[0]

print("Nombre de lignes avec des valeurs manquantes :", nombre_lignes_manquantes)

In [ ]:
# Analyse de correlation
corr_matrix = data.corr()
plt.figure(figsize=(20, 16))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm')
plt.title('Matrice de correlation')
plt.show()

In [ ]:
# Selection des variables les plus correlees avec is_fraud
correlations_with_fraud = corr_matrix['is_fraud'].abs().sort_values(ascending=False)
print("Top 10 variables les plus corrélées avec is_fraud:")
print(correlations_with_fraud.head(6))

nous utiliserons les régions comme localisation par la suite

In [ ]:
# Information mutuelle
X = data.drop(columns=['is_fraud']) #Contient les variables explicatives (indépendantes)
y = data['is_fraud'] #Contient la variable cible (dépendante)
mi_scores = mutual_info_regression(X, y)
mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)
print("\nTop 10 variables selon l'information mutuelle:")
print(mi_scores.head(5))

In [ ]:
# Sélection finale des variables importantes
important_features = list(set(list(correlations_with_fraud.head(5).sort_values(ascending=False).index) + list(mi_scores.head(5).sort_values(ascending=False).index)))
important_features = [f for f in important_features if f != 'is_fraud']
print("\nVariables importantes sélectionnées:")
print(len(important_features))

fusionner les dataframes

In [ ]:
# Creation du dataset final
X_final = data[important_features]
y_final = data['is_fraud']

In [ ]:
# Entrainement du modele avec Isolation Forest
model = IsolationForest(contamination=0.5, random_state=42)
y_pred = model.fit_predict(X_final)
y_pred

In [ ]:
# evaluons la precision du modele
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Convertir les prédictions de l'Isolation Forest (-1 pour anomalie, 1 pour normal) 
# en format binaire (1 pour anomalie, 0 pour normal)
#y_pred_binary = [1 if pred == -1 else 0 for pred in y_pred]

y_pred_binary = np.where(y_pred==-1, 0, y_pred)

# Calculer et afficher les métriques de classification
print(classification_report(y_final, y_pred_binary))

# Calculer l'AUC-ROC
auc_roc = roc_auc_score(y_final, y_pred_binary)
print(f"AUC-ROC: {auc_roc}")

In [ ]:
print(y_pred_binary[:20])
print(y_pred[:20])

In [ ]:
# Afficher la matrice de confusion
print("Matrice de confusion:")
print(confusion_matrix(y_final, y_pred_binary))

In [ ]:
# Affichage des résultats
print("\nNombre d'anomalies détectées:", sum(y_pred == -1))

In [ ]:
X_final

**Feature selection**

In [ ]:
test = SelectKBest(score_func=chi2)
fit = test.fit(data.drop(columns="is_fraud").abs(), list(y_final))

In [ ]:
np.set_printoptions(precision=3)
features = fit.transform(data.drop(columns="is_fraud"))
print(features[0:5, :])
feat = pd.DataFrame()
feat["num_features"] = data.drop(columns="is_fraud").columns
feat["score"] = fit.scores_
feat.sort_values(by="score", ascending=False, inplace=True)
feat = feat.reset_index().drop(columns=["index"])
feat

In [ ]:

def model_fit_transform(features):
    X  = data[features]
    model_1 = IsolationForest(contamination=0.5, random_state=42)
    model_1.fit(X)

    pred_1= model_1.predict(X)
    pred_1 = np.where(pred_1==1, 0, pred_1)
    pred_1 = np.where(pred_1==-1, 1, pred_1)
    print(f"{pred_1[:10]}\n\n")

    #evaluation
    print(roc_auc_score(y_final, pred_1))

In [ ]:
feature_imp = list(feat.head(4).num_features)
print(len(important_features))
print(len(feature_imp))

In [ ]:
model_fit_transform(important_features)

In [ ]:
model_fit_transform(feature_imp)

In [ ]:
len(list(data.columns))

## Classification des customers par cluster

In [ ]:
customers.head()

In [ ]:
customers["region_e"] = le.fit_transform(customers.region)
customers.head()

In [ ]:
data.head()

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(data.drop(columns=["anomalie", "transaction_id", "customer_id", "email", "transaction_date", "phone_number"]))

In [ ]:
kmeans = KMeans(random_state=42)
kmeans.fit_transform(X_pca)

In [ ]:
labels = kmeans.labels_
centers = kmeans.cluster_centers_

In [ ]:
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=kmeans.labels_, cmap='viridis')

In [ ]:
from sklearn.model_selection import train_test_split  
from sklearn.ensemble import RandomForestClassifier   # Importation du classificateur RandomForest
from sklearn.metrics import classification_report, accuracy_score 

In [ ]:
# Connexion à la base de données SQLite
conn = sqlite3.connect('../Data/db/fraude_detection_warehouse_.db')

In [ ]:
# Étape 1: Chargement les données de la table transactions
data = pd.read_sql("SELECT * FROM transactions", conn)

In [ ]:
X = data.drop('is_fraud', axis=1)   # Suppression de la colonne is_fraud' pour obtenir les caractéristiques
y = data['is_fraud'] # La cible est la colonne 'is_fraud'
y

In [ ]:

# Conversion de la colonne 'transaction_date' en type datetime
X['transaction_date'] = pd.to_datetime(X['transaction_date'], errors='coerce')

# Extraire jour, mois, et année de la colonne 'transaction_date'
X['jour'] = X['transaction_date'].dt.day
X['mois'] = X['transaction_date'].dt.month
X['annee'] = X['transaction_date'].dt.year

# Supprimer la colonne 'transaction_date'
X = X.drop(columns=['transaction_date'])

# Vérifier les changements
print(X.dtypes)

In [ ]:
# Supprimer les colonnes ID qui ne sont pas des features pertinentes
X = X.drop(columns=['transaction_id', 'customer_id', 'device_id'])

In [ ]:
print(X.dtypes)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Liste des colonnes catégorielles à encoder
cols_to_encode = ['transaction_type', 'location', 'status', ]

# Encoder les colonnes en valeurs numériques
label_encoder = LabelEncoder()
for col in cols_to_encode:
    X[col] = label_encoder.fit_transform(X[col].astype(str))

# Vérifier les changements
print(X.dtypes)

In [ ]:
# Étape 3: Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# `test_size=0.2` signifie que 20% des données seront utilisées pour le test, 80% pour l'entraînement
# `random_state=42` est un paramètre pour reproduire les résultats aléatoires

In [ ]:
# Créer et entraîner le modèle RandomForest
modele = RandomForestClassifier(n_estimators=100, random_state=42)
modele.fit(X_train, y_train)  # Entraînement du modèle sur les données d’entraînement

In [ ]:
y_pred = modele.predict(X_test)  # Prédiction des données de test

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

# Calculer la précision
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision (Accuracy) : {accuracy:.4f}")

# Afficher la matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matrice de confusion :\n", conf_matrix)

# Afficher un rapport de classification détaillé (précision, rappel, F1-score)
print("Rapport de classification :\n", classification_report(y_test, y_pred))

# Calculer le score F1
f1 = f1_score(y_test, y_pred)
print(f"Score F1 : {f1:.4f}")

INTERPRETATION: notre modele a 62,63 % de predictions correctes

# LE BAGGING

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# CREATION DE NOTRE MODELE DE BAGGING

# Définissons le modèle de base (RandomForestClassifier)
base_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=15)

# Création du BaggingClassifier avec notre modèle de base
bagging_model = BaggingClassifier(estimator=base_model, 
                                  n_estimators=10,   # Nombre de modèles à entraîner
                                  random_state=42,
                                  n_jobs=-1)  # Utilisation de tous les cœurs CPU disponibles

In [ ]:
# Entraînons le modèle  sur nos données d'entraînement
bagging_model.fit(X_train, y_train)

In [ ]:
#  prédictions sur les données de test
y_pred = bagging_model.predict(X_test)

# Calcul de la précision (Accuracy)
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision (Accuracy) : {accuracy:.4f}")

#  matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matrice de confusion :")
print(conf_matrix)

# rapport de classification
class_report = classification_report(y_test, y_pred)
print("Rapport de classification :")
print(class_report)

INTERPRETATION: notre modele a 65,38 % de predictions correctes, soit 2.75% de plus que notre modele de classification.

# LE BOOSTING

RECHERCHE DES MEILLEURS HYPERPARAMETRES

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from scipy.stats import uniform, randint

# Définir la distribution des paramètres à tester
param_dist = {
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'n_estimators': randint(100, 500),
    'subsample': uniform(0.7, 0.3),
    'colsample_bytree': uniform(0.7, 0.3),
    'gamma': uniform(0, 0.5)
}

# Créer le modèle XGBoost
xgb_model = xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False)

# Configurer RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, n_iter=50, cv=3, scoring='f1', verbose=1, n_jobs=-1, random_state=42)

# Exécuter la recherche aléatoire
random_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres
print(f"Meilleurs paramètres trouvés : {random_search.best_params_}")

In [ ]:
# Créer un modèle XGBoost avec les meilleurs hyperparamètres trouvés

best_xgb_model = xgb.XGBClassifier(
    colsample_bytree=0.7816396748153905,
    gamma=0.32384506027068116,
    learning_rate=0.010156113098594747,
    max_depth=7,
    n_estimators=332,
    subsample=0.7914343774474086,
    objective='binary:logistic',
    use_label_encoder=False,
    random_state=42
)

# Entraîner le modèle sur les données d'entraînement
best_xgb_model.fit(X_train, y_train)

# Prédire les étiquettes pour les données de test
y_pred = best_xgb_model.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Précision (Accuracy): {accuracy:.4f}")
print(f"Score F1: {f1:.4f}")
print("Matrice de confusion :\n", confusion)
print("Rapport de classification :\n", classification_rep)

la precision de notre modele de boosting est de 66,29%, soit 1% plus precis que le bagging.

Cherchons a comprendre si c'est la technique qui est meilleure pour notre cas ou alors si c'est grace à l'utilisation des meilleurrs hyperparametres que nous avons de meilleurs resultats.

### BAGGING AVEC LES HYPERPARAMETRES TROUVES

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# Créer le classifieur de base avec les hyperparamètres optimisés
base_classifier = DecisionTreeClassifier(
    max_depth=7,
    random_state=42
)

# Créer le modèle Bagging avec les paramètres optimisés
bagging_model = BaggingClassifier(
    estimator=base_classifier,
    n_estimators=332,  # équivalent à n_estimators dans XGBoost
    max_samples=0.7914343774474086,  # équivalent à subsample dans XGBoost
    random_state=42
)

# Entraîner le modèle sur les données d'entraînement
bagging_model.fit(X_train, y_train)

# Prédire les étiquettes pour les données de test
y_pred_bagging = bagging_model.predict(X_test)

# Évaluer les performances du modèle
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
f1_bagging = f1_score(y_test, y_pred_bagging)
confusion_bagging = confusion_matrix(y_test, y_pred_bagging)
classification_rep_bagging = classification_report(y_test, y_pred_bagging)

print(f"Précision (Accuracy): {accuracy_bagging:.4f}")
print(f"Score F1: {f1_bagging:.4f}")
print("Matrice de confusion :\n", confusion_bagging)
print("Rapport de classification :\n", classification_rep_bagging)

## INTERPRETATION
PREDICTIONS CORRECTES: 66,36%

Vrais négatifs (TN) : 5661 / non fraudes classées comme non fraudes Faux positifs (FP) : 5148 / non fraudes classées comme fraude Faux négatifs (FN) : 3263 / fraudes classés comme non fraudes Vrais positifs (TP) : 10 928 / fraudes classées comme fraude
FRAUDES DETECTEES: 10 928 et FRAUDES NON DETECTEES: 3263

63% de prediction correctes sur la classe de non fraude et 68% de predictions correctes sur la classe des fraudes
(le f1 score nous montre que notre modele a mieux detecter les cas de fraudes que de non fraude)

- conclusion:

l'utilisaton des meilleurs hyperparametres couplé avec un arbre de decision comme modele de base a amelioré la precision de notre modele de Bagging de 1% par rapport au modele de bagging utilisant des hyperparametres aleatoires .
En comparant les performances de nos methodes (bagging et boosting) couplés avec l'utilisation des hyperparametres trouvés, on se rend compte que les resultats en utilisant le bagging sont pratiquement egales(0.07% de difference). On conclut que utiliser un modèle d'ensemble a amélioré nos resultats. Cependant, le choix parmi les deux modèles d'ensemble a eu peu d'influence sur nos resultats mais utiliser les meilleurs hyperparametres a ete le facteur ayant le plus amelioré les resultats de nos 2 modèles

In [ ]:
import joblib

# Entraînement du modèle (par exemple, un modèle RandomForest)
# modèle = RandomForestClassifier(...)
# modèle.fit(X_train, y_train)

# Enregistrer le modèle entraîné
joblib.dump(bagging_model, '../modeles/bagging_model.pkl')

print("Le modèle a été enregistré avec succès.")